In [15]:
import string
import random
import numpy as np
import pandas as pd
from collections import defaultdict, deque
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
data=pd.read_csv('processed.csv')
data.head()

,password,strength,length,capital,small,numeric,special
0,kzde5577,1,8,0,4,4,0
1,kino3434,1,8,0,4,4,0
2,visi7k1yr,1,9,0,7,2,0
3,megzy123,1,8,0,5,3,0
4,lamborghin1,1,11,0,10,1,0


In [4]:
y=data['strength'].values
x=data[['length','capital','small','special','numeric']].values

In [5]:
xtrain,xtest,ytrain,ytest=tts(x,y,test_size=0.2,random_state=42)

sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)

model=RandomForestClassifier(n_estimators=100,random_state = 42)
model.fit(xtrain,ytrain)

RandomForestClassifier(random_state=42)

In [6]:
ypred=model.predict(xtest)

In [11]:
print("The accuracy of the model is: ",accuracy_score(ytest,ypred)*100," % !!!")
print(classification_report(ytest,ypred))
print("The confusion matrix is: ")
cf=confusion_matrix(ytest,ypred)
print(cf)

The accuracy of the model is:  99.9626665073771  % !!!
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17908
           1       1.00      1.00      1.00     99519
           2       1.00      1.00      1.00     16501

    accuracy                           1.00    133928
   macro avg       1.00      1.00      1.00    133928
weighted avg       1.00      1.00      1.00    133928

The confusion matrix is: 
[[17865    43     0]
 [    7 99512     0]
 [    0     0 16501]]


In [19]:
import random
import string
import numpy as np
from collections import defaultdict, deque

class PasswordGenerator:
    def __init__(self, password_length, charset=string.ascii_letters + string.digits + string.punctuation, epsilon=0.5, epsilon_min=0.01, epsilon_decay=0.999, gamma=0.95, alpha=1e-5, batch_size=32, memory_size=1000):
        self.password_length = password_length
        self.charset = charset
        self.q_table = defaultdict(lambda: np.zeros(len(self.charset)))
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.alpha = alpha
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)
        
    def act(self, state):
        q_values = self.q_table[state]
        exp_q_values = np.exp(q_values / self.epsilon)
        probabilities = exp_q_values / np.sum(exp_q_values)
        probabilities = np.array(probabilities)  
        return self.charset[np.random.choice(len(self.charset), p=probabilities)]

    def update_q_table(self, state, action, reward, next_state):
        action_index = self.charset.index(action)
        current_q = self.q_table[state][action_index]
        target_q = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action_index] = current_q + self.alpha * (target_q - current_q)
        
    def get_reward(self, guess, target_password):
        if guess == target_password:
            return 1
        else:
            num_letters = sum(1 for char in guess if char in string.ascii_letters)
            num_uppercase = sum(1 for char in guess if char in string.ascii_uppercase)
            num_lowercase = sum(1 for char in guess if char in string.ascii_lowercase)
            num_special = sum(1 for char in guess if char in string.punctuation)
            num_digits = sum(1 for char in guess if char in string.digits)

            reward = -0.1
            if num_letters == self.password_length:
                reward += 0.2
            if num_uppercase >= 1:
                reward += 0.2
            if num_lowercase >= 1:
                reward += 0.2
            if num_special >= 1:
                reward += 0.2
            if num_digits >= 1:
                reward += 0.1

            return reward
    
    def train(self, passwords_list, num_episodes, checkpoint=100):
        for episode in range(num_episodes):
            target_password = random.choice(passwords_list)
            state = ''
            for _ in range(self.password_length):
                action = self.act(state)
                reward = self.get_reward(state + action, target_password)
                next_state = state + action
                self.memory.append((state, action, reward, next_state, next_state == target_password))
                self.update_q_table(state, action, reward, next_state)
                state = next_state

            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

            if (episode + 1) % checkpoint == 0:
                print(f"Episode {episode + 1}/{num_episodes}, Epsilon: {self.epsilon:.3f}")

    def generate_password(self):
        candidate_passwords = []
        for _ in range(5):  
            state = ''
            for _ in range(self.password_length):
                action = self.act(state)
                state += action
            candidate_passwords.append(state)

        return random.choice(candidate_passwords)



In [20]:
passwords_list = data['password'].tolist()
password_generator = PasswordGenerator(password_length=15)
password_generator.train(passwords_list, num_episodes=10000)

Episode 100/10000, Epsilon: 0.452
Episode 200/10000, Epsilon: 0.409
Episode 300/10000, Epsilon: 0.370
Episode 400/10000, Epsilon: 0.335
Episode 500/10000, Epsilon: 0.303
Episode 600/10000, Epsilon: 0.274
Episode 700/10000, Epsilon: 0.248
Episode 800/10000, Epsilon: 0.225
Episode 900/10000, Epsilon: 0.203
Episode 1000/10000, Epsilon: 0.184
Episode 1100/10000, Epsilon: 0.166
Episode 1200/10000, Epsilon: 0.151
Episode 1300/10000, Epsilon: 0.136
Episode 1400/10000, Epsilon: 0.123
Episode 1500/10000, Epsilon: 0.111
Episode 1600/10000, Epsilon: 0.101
Episode 1700/10000, Epsilon: 0.091
Episode 1800/10000, Epsilon: 0.083
Episode 1900/10000, Epsilon: 0.075
Episode 2000/10000, Epsilon: 0.068
Episode 2100/10000, Epsilon: 0.061
Episode 2200/10000, Epsilon: 0.055
Episode 2300/10000, Epsilon: 0.050
Episode 2400/10000, Epsilon: 0.045
Episode 2500/10000, Epsilon: 0.041
Episode 2600/10000, Epsilon: 0.037
Episode 2700/10000, Epsilon: 0.034
Episode 2800/10000, Epsilon: 0.030
Episode 2900/10000, Epsilon: 

In [24]:
#Testing accuracy of RL model

num_pass = 1000
correct_predictions = 0
for _ in range(num_pass):
    generated_password = password_generator.generate_password()
    password_features = np.array([len(generated_password),
                                  sum(1 for char in generated_password if char in string.ascii_uppercase),
                                  sum(1 for char in generated_password if char in string.ascii_lowercase),
                                  sum(1 for char in generated_password if char in string.punctuation),
                                  sum(1 for char in generated_password if char in string.digits)])
    password_features = sc.transform([password_features])
    predicted_class = model.predict(password_features)[0]
    
    

    if predicted_class == 0 and len(generated_password) < 8:
        correct_predictions += 1
    elif predicted_class == 1 and (len(generated_password) >= 8 and len(generated_password) < 12):
        correct_predictions += 1
    elif predicted_class == 2 and len(generated_password) >= 12:
        correct_predictions += 1
        
accuracy = correct_predictions / num_pass
print(f"Accuracy of the RL model: {accuracy:.2f}")

Accuracy of the RL model: 1.00
